# Content Based Filtering
- Content Based Filtering (CBF) hanya membutuhkan single user, tidak seperti collaborative filtering yang membutuhkan banyak user untuk dibandingkan.
- Cara kerja algoritma CBF adalah mencari sesuatu dalam kasus ini adalah film yang sudah user tonton. lalu mesin akan mencari film - film yang mirip dengan film yang sudah ditonton oleh user. Anggap saja user sudah menonton film AADC, lalu mesin mendeteksi bahwa film AADC itu mirip dengan film Dilan 1990. Maka mesin akan merekomendasikan film Dilan 1990 kepada user.

## Case Story
- Rangga, seorang data scientist di netclick.com mendapatkan tugas baru dari managernya yaitu Asep.
- Ucup merupakan tetangga Asep dan Ucup dan ingin melihat film yang bagus sesuai dengan seleranya. Lalu si Asep meminta rangga untuk menganalisa film apa yang cocok untuk si Ucup berdasarkan history film yang pernah ucup tonton.
- Asep menantang Ucup untuk menggunakan algoritma CBF.
- Dara dataset yang diberikan Asep, Ucup termasuk kedalam user yang mempunyai id 352.

## Load Library

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

## Load Datasets
- Dataset yang digunakan berasal dari MovieLens dengan 2 datasets yang akan kita gunakan yaitu data ratings dan movies.

In [3]:
movies = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\movies.csv')
ratings = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\ratings.csv')

## Data Cleaning

In [4]:
#Menghapus tahun di title 
movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')

#Menghapus whitespace
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [5]:
#Split genres karena akan digunakan di algoritma CBF
movies['genres'] = movies['genres'].str.split('|')

In [6]:
#Drop kolom timestamp
ratings = ratings.drop(columns=['timestamp'])

## Modelling

In [7]:
def rekomendasi_film_content(user_id):
    #Melakukan One Hot Encoding untuk dataframe baru
    #Kita copy dulu
    new_movies = movies.copy()
    #Kita looping untuk setiap row di df, lalu looping lagi di genres list dan taroh list tsb ke dalam kolom
    for index, row in movies.iterrows():
        for genre in row['genres']:
            new_movies.at[index, genre] = 1    
    #Yang tidak termasuk genrenya ke dalam list maka kita diisi dengan 0
    new_movies = new_movies.fillna(0)
    
    #Kita membuat movie input dari user
    rating_grouping = ratings.groupby('userId')
    user_id_film = rating_grouping.get_group(user_id).head(5)
    
    #Kemudian melakukan One Hot Encoding untuk input movie user
    user_movie_encoding = new_movies[new_movies['movieId'].isin(user_id_film['movieId'].tolist())]
    #Reset index
    user_movie_encoding = user_movie_encoding.reset_index(drop=True)
    #Drop unused columns
    user_movie_encoding = user_movie_encoding.drop(columns=['movieId', 'title', 'genres'])
    
    #Mencari tahu user profile berdasarkan genrenya
    #Menggunakan dot product untuk mendapatkan bobot dari setiap genre
    user_profile = user_movie_encoding.transpose().dot(user_id_film['rating'].reset_index(drop=True))
    
    #Melakukan Rekomendasi Film
    new_movies.set_index(new_movies['movieId'], inplace=True)
    new_movies = new_movies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
    #Multiply the genres by the weights and then take the weighted average
    rekomendasi_film = ((new_movies*user_profile).sum(axis=1))/(user_profile.sum())
    #Urutkan berdasarkan nilai weughted average tertinggti
    rekomendasi_film = rekomendasi_film.sort_values(ascending=False)
    #Rekomendasi film final untuk User
    rekomendasi_film_final = movies.loc[movies['movieId'].isin(rekomendasi_film.head(10).keys())]
    
    return rekomendasi_film_final

In [10]:
user = int(input('Masukan User Id: '))
rekomendasi_film_content(user)

Masukan User Id: 352


,movieId,title,genres
167,198,Strange Days,"[Action, Crime, Drama, Mystery, Sci-Fi, Thriller]"
1330,1799,Suicide Kings,"[Comedy, Crime, Drama, Mystery, Thriller]"
4940,7445,Man on Fire,"[Action, Crime, Drama, Mystery, Thriller]"
5327,8860,Cellular,"[Action, Crime, Drama, Mystery, Thriller]"
5556,26701,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M..."
5672,27674,11:14,"[Comedy, Crime, Drama, Mystery, Thriller]"
5808,31921,"Seven-Per-Cent Solution, The","[Adventure, Comedy, Crime, Drama, Mystery, Thr..."
7372,79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill..."
7441,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film..."
9411,165347,Jack Reacher: Never Go Back,"[Action, Crime, Drama, Mystery, Thriller]"


- Jadi setelah melakukan analisis, Rangga memberikan 10 top film untuk Ucup adalah sebagai berikut:
    1. Strange Days
    2. Suicide Kings
    3. Man on Fire
    4. Cellular	
    5. Patlabor: The Movie
    6. 11:14
    7. Seven-Per-Cent Solution, The
    8. Rubber	
    9. Inception		
    10. Jack Reacher: Never Go Back	